In [19]:
import os
import json
import time
import pyautogui

def get_key():
    with open("secretkey.json", "r") as json_file:
        return json.load(json_file)["api_key"]

try:
    pyautogui.getWindowsWithTitle("my_presentation.pptx")[0].close()
    time.sleep(0.1)

except IndexError:
    print("Window '.pptx' not found.")

Window '.pptx' not found.


In [20]:
sys_prompt = """Instruct:
You are to use the provided infomation to generate content for a powerpoint slide.


Guideline:
return using a json format


Example:
{
"title": "[title, subtitle]"

}
"""

In [21]:
import os
import requests
import base64

# Configuration
GPT4V_KEY = get_key()

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information.",
                }
            ],
        }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 1000,
}

GPT4V_ENDPOINT = "https://intern2024openai.openai.azure.com/openai/deployments/gpt4-Omni/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

print(response.json()["choices"][0]["message"]["content"])

Sure, I'd be happy to help! What information are you looking for today?


In [22]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor


def remove_all_placeholders(slide):
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)


def add_title_slide(prs, title_text, subtitle_text):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add title text box
    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(2), slide_width - Inches(2), Inches(1.5)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(54)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subtitle text box
    subtitle_shape = slide.shapes.add_textbox(
        Inches(1), Inches(4), slide_width - Inches(2), Inches(1)
    )
    subtitle_frame = subtitle_shape.text_frame
    subtitle_frame.text = subtitle_text
    subtitle_frame.paragraphs[0].font.size = Pt(32)
    subtitle_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subtitle_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105) 


def add_table_of_contents_slide(prs, title_text, num_sections, sections):
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    remove_all_placeholders(slide)

    title_shape = slide.shapes.add_textbox(
        Inches(0.5), Inches(0.5), Inches(12.33), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True

    # Constants for positioning
    left_positions = [Inches(0.5), Inches(6.5)]  # Adjusted to move more to the left
    top_start = Inches(1.5)
    space_between = Inches(1.5)
    max_per_column = 4

    # Function to adjust text size if overflowing
    def adjust_text_size(text_frame, max_width):
        for paragraph in text_frame.paragraphs:
            run = paragraph.runs[0]
            while (
                run.font.size is None or run.font.size.pt > 10
            ):  # Set a minimum font size
                if run.font.size is not None and text_frame.auto_size:
                    break
                if run.font.size is None:
                    run.font.size = Pt(20)  # Start with 20 if size is None
                else:
                    run.font.size -= Pt(1)  # Decrease font size by 1pt

    for i, (heading, subheading) in enumerate(sections):
        column = i // max_per_column
        row = i % max_per_column

        # Calculate positions
        left = left_positions[column]
        top = top_start + row * space_between

        # Add section number
        number_shape = slide.shapes.add_textbox(left, top, Inches(1.0), Inches(1.0))
        number_frame = number_shape.text_frame
        number_frame.text = f"{i + 1:02}"
        number_frame.paragraphs[0].font.size = Pt(40)
        number_frame.paragraphs[0].font.bold = True
        number_frame.paragraphs[0].font.color.rgb = RGBColor(89, 89, 89)

        # Add section heading
        heading_shape = slide.shapes.add_textbox(
            left + Inches(1.0),
            top,
            Inches(5.0),
            Inches(0.5),  # Moved closer to the left
        )
        heading_frame = heading_shape.text_frame
        heading_frame.text = heading
        heading_frame.paragraphs[0].font.size = Pt(20)
        heading_frame.paragraphs[0].font.bold = True
        adjust_text_size(heading_frame, Inches(5.0))

        # Add section subheading with wrapping text
        subheading_shape = slide.shapes.add_textbox(
            left + Inches(1.0),
            top + Inches(0.5),
            Inches(5.0),
            Inches(1.0),  # Moved closer to the left
        )
        subheading_frame = subheading_shape.text_frame
        subheading_frame.text = subheading
        subheading_frame.word_wrap = True  # Enable word wrapping
        subheading_frame.paragraphs[0].font.size = Pt(14)
        subheading_frame.paragraphs[0].font.color.rgb = RGBColor(128, 128, 128)
        adjust_text_size(subheading_frame, Inches(5.0))


def add_section_slide(prs, section_number, heading, subheading):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add section number
    number_shape = slide.shapes.add_textbox(
        Inches(4.5), Inches(1.5), Inches(4.0), Inches(1.0)
    )
    number_frame = number_shape.text_frame
    number_frame.text = f"{section_number:02}"
    number_frame.paragraphs[0].font.size = Pt(100)
    number_frame.paragraphs[0].font.bold = True
    number_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add heading text box
    heading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(3.5), slide_width - Inches(2), Inches(1.5)
    )
    heading_frame = heading_shape.text_frame
    heading_frame.text = heading
    heading_frame.paragraphs[0].font.size = Pt(54)
    heading_frame.paragraphs[0].font.bold = True
    heading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subheading text box
    subheading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(5), slide_width - Inches(2), Inches(1)
    )
    subheading_frame = subheading_shape.text_frame
    subheading_frame.text = subheading
    subheading_frame.paragraphs[0].font.size = Pt(32)
    subheading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subheading_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105)


def add_content_slide(prs, title, points):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add title text box
    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(0.5), slide_width - Inches(2), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    num_points = len(points)
    assert 2 <= num_points <= 5, "Number of points must be between 2 and 5"

    top_start = Inches(2)
    left_margin = Inches(1)
    col_width = (slide_width - 2 * left_margin) / 2

    if num_points == 2:
        top = top_start
        for i, (header, content) in enumerate(points):
            left = left_margin + i * col_width

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(0.5), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 3:
        top = top_start
        col_width = (slide_width - 2 * left_margin) / 3
        for i, (header, content) in enumerate(points):
            left = left_margin + i * col_width

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top+ Inches(0.75), col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(1.25), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 4:
        top_positions = [top_start, top_start + Inches(2.5)]
        for i, (header, content) in enumerate(points):
            left = left_margin + (i % 2) * col_width
            top = top_positions[i // 2]

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(0.5), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 5:
        top_positions = [top_start, top_start + Inches(3.5)]
        for i, (header, content) in enumerate(points):
            if i == 2:  # Middle item for 5 points
                left = (slide_width - col_width) / 2
                top = top_start + Inches(1.75)
            else:
                left = left_margin + (i % 2) * col_width
                if i < 2:
                    top = top_positions[0]
                else:
                    top = top_positions[1]

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                    left, top + Inches(0.5), col_width, Inches(1.5)
                )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


def add_side_by_side_text_slide(prs, heading, left_text, right_text):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add heading text box
    heading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(0.5), slide_width - Inches(2), Inches(1)
    )
    heading_frame = heading_shape.text_frame
    heading_frame.text = heading
    heading_frame.paragraphs[0].font.size = Pt(32)
    heading_frame.paragraphs[0].font.bold = True
    heading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Calculate positions for the two text boxes
    col_width = (slide_width - 3 * Inches(1)) / 2
    left_margin = Inches(1)
    top_margin = Inches(1.5)

    # Add left text box
    left_shape = slide.shapes.add_textbox(
        left_margin, top_margin + Inches(1), col_width, slide_height - Inches(2.5)
    )
    left_frame = left_shape.text_frame
    left_frame.text = left_text
    left_frame.word_wrap = True
    left_frame.paragraphs[0].font.size = Pt(18)
    left_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    # Add right text box
    right_shape = slide.shapes.add_textbox(
        left_margin + col_width + Inches(1),
        top_margin + Inches(1),
        col_width,
        slide_height - Inches(2.5),
    )
    right_frame = right_shape.text_frame
    right_frame.text = right_text
    right_frame.word_wrap = True
    right_frame.paragraphs[0].font.size = Pt(18)
    right_frame.paragraphs[0].alignment = PP_ALIGN.LEFT




def main():
    
    prs = Presentation()

    # Set slide width and height for 16:9 aspect ratio
    prs.slide_width = Inches(13.33)
    prs.slide_height = Inches(7.5)

    # Add a title slide
    add_title_slide(prs, "Hello, Pookie😘", "This is a subtitle in your first slide")

    sections = [
        ("Name of the Section 1", "You can describe the topic of the section here"),
        ("Name of the Section 2", "You can describe the topic of the section here"),
        ("Name of the Section 3", "You can describe the topic of the section here"),
        ("Name of the Section 4", "You can describe the topic of the section here"),
        ("Name of the Section 5", "You can describe the topic of the section here"),
        ("Name of the Section 6", "You can describe the topic of the section here"),
        ("Name of the Section 7", "You can describe the topic of the section here"),
        ("Name of the Section 8", "You can describe the topic of the section here"),
    ]
    print(type(sections))
    # Add a table of contents slide
    add_table_of_contents_slide(prs, "Table of Contents", len(sections), sections)

    for i, (heading, subheading) in enumerate(sections):
        add_section_slide(prs, i + 1, heading, subheading)

    points = [
        ("Mercury", "Mercury is the closest planet to the Sun and the smallest one in the Solar System."),
        ("Venus", "Venus has a beautiful name and is the second planet from the Sun."),
    ]
    add_content_slide(prs, "Maybe You Need to Divide the Content", points)

    points = [
        ("Mercury", "Mercury is the closest planet to the Sun and the smallest of them all."),
        ("Venus", "Venus has a beautiful name and is the second planet from the Sun."),
        ("Mars", "Despite being red, Mars is actually a cold place. It’s full of iron oxide dust."),
    ]
    add_content_slide(prs, "Here Are Three Important Ideas", points)

    points = [
        ("Mars", "Mars is actually a very cold place."),
        ("Jupiter", "Jupiter is the biggest planet of them all."),
        ("Venus", "Venus has extremely high temperatures."),
        ("Saturn", "Saturn is a gas giant and has several rings."),
    ]
    add_content_slide(prs, "Divide the Content in Four Ideas", points)

    points = [
        ("Mars", "Mars is actually a very cold place."),
        ("Jupiter", "Jupiter is the biggest planet of them all."),
        ("Venus", "Venus has extremely high temperatures."),
        ("Saturn", "Saturn is a gas giant and has several rings."),
        ("Saturn", "Saturn is a gas giant and has several rings."),
    ]
    add_content_slide(prs, "Divide the Content in Five Ideas", points)
    add_side_by_side_text_slide(prs, "Do You Need Longer Text?", 
                                "Mercury is the closest planet to the Sun and the smallest one in the entire Solar System. This planet’s name has nothing to do with the liquid metal, since Mercury was named after the Roman messenger god. Despite being closer to the Sun than Venus, its temperatures aren’t as terribly hot as that planet’s. Its surface is quite similar to that of Earth’s Moon, which means there are a lot of craters and plains.",
                                "Speaking of craters, many of them were named after artists or authors who made significant contributions to their respective fields. Mercury takes a little more than 58 days to complete its rotation, so try to imagine how long days must be there! Since the temperatures are so extreme, albeit not as extreme as on Venus, and the solar radiation is so high, Mercury has been deemed to be non-habitable for humans") 
    prs.save("my_presentation.pptx")

    print("Presentation created successfully")


if __name__ == "__main__":
    main()

<class 'list'>
Presentation created successfully


In [23]:
os.startfile("my_presentation.pptx")